## Collisions in the Corona and the Solar Wind

Our formula for the collision frequency between electrons and ions was previously:
$$ \nu = n\sigma_c v \ln \Lambda,$$

where $n$ is the density, $\sigma_c$ is the cross section for the interaction, $v$ is the average relative velocity, and $\ln\Lambda$ is the Coulomb logarithm. (See Chapter 3 Section....for a detailed review of collision frequency.)

There are two functions in Plasmapy for calculating the collision frequency. One, SingleParticleCollisionFrequencies() uses the relative drift velocity, and the other, MaxwellianCollisionFrequencies() uses the relative thermal speed. Both assume 2 kinds of particles, $a$ and $b$, but the first treats it as a collision between two individual particles, while the other treats it as two interacting fluids. The Maxwellian approximation is appropriate when the fluids are "slowly flowing," which means
$$v_{drift} \ll \sqrt{v_a^2 + v_b^2},$$

where $v_{drift}$ is the relative drift between the two particles, while $v_a$ and $v_b$ are the thermal velocities of the fluids. Let's check which one would be valid for approximating the collision frequency in the solar corona and wind.

In [65]:
%matplotlib inline

import plasmapy.formulary.collisions as coll
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.constants import hbar as ℏ
from astropy.visualization import quantity_support

from plasmapy.formulary.collisions import (
    Coulomb_logarithm,
    impact_parameter,
    impact_parameter_perp,
)
from plasmapy.formulary.speeds import thermal_speed
from plasmapy.particles import reduced_mass

[thermal_speed()]:https://docs.plasmapy.org/en/stable/notebooks/formulary/thermal_speed.html

The [thermal_speed()] function can calculate the thermal speed (most probable), the rms speed, and the mean speed. In this case, we will use the 'most_probable' method. (See Ch. 1 Section Thermodynamics for a more detailed description) Note that the temperature and density of the electrons can and often does differ from the temperature and density of the protons in solar environments. But for simplicity, and to gain an order-of-magnitude estimate, we consider them the same here.

In [66]:
T_corona = 1e6 * u.K
v_e,v_p = thermal_speed(T=T_corona, particle=["e-","p+"], method="most_probable", ndim=3)
print(v_e,v_p)

5505694.902726359 m / s 128486.5732808313 m / s


Those are some very fast thermal velocities!

The equation for drift velocity is  
$$v_{drift} = \frac{\mathbf{E}\times \mathbf{B}}{B^2}.$$
So when the strength of the electric field $\mathbf{E}$ is much smaller than the magnetic field $\mathbf{B}$, the drift velocity will be small. This is the case for the corona and the solar wind, which are highly magnetized plasmas. We then can safely assume that our "slowly flowing" condition is met and will use the Maxwellian method for the collision frequency.

[Debye length]: https://en.wikipedia.org/wiki/Debye_length
[reduced mass]: https://en.wikipedia.org/wiki/Reduced_mass
[de Broglie wavelength]: https://en.wikipedia.org/wiki/Matter_wave
[Debye_length()]: https://docs.plasmapy.org/en/stable/api/plasmapy.formulary.lengths.Debye_length.html
[impact_parameter_perp()]: https://docs.plasmapy.org/en/latest/api/plasmapy.formulary.collisions.lengths.impact_parameter_perp.html#impact-parameter-perp
[impact_parameter()]: https://docs.plasmapy.org/en/latest/api/plasmapy.formulary.collisions.lengths.impact_parameter.html
[plasmapy.formulary]: https://docs.plasmapy.org/en/latest/formulary/index.html

We know $v$, and now must calculate $\ln\Lambda$. The Coulomb logarithm accounts for the range in impact parameters for the different collisions, and is given by 
$$\ln{Λ} = \ln\left(\frac{b_\max}{b_\min}\right).$$

The minimum and maximum impact parameters ($b_\min$ and $b_\max$, respectively) represent the range of distances of closest approach. Usually $b_\max$ is given by the [Debye length], 
$$λ_D = \sqrt{\frac{\epsilon_0 kT_e}{n_eq_e^2}},$$
which can be calculated with [Debye_length()], but is also built into the [impact_parameter()] function. 

The inner impact parameter $b_\min$ could be set to a 90° deflection angle, $ρ_⟂$, which can be calculated with [impact_parameter_perp()].  Alternatively, $b_\min$ could be set to be the [de Broglie wavelength], $λ_{dB} = h/\mu v$, calculated using the [reduced mass], $μ$, of the particles. Typically, we choose whichever value is larger, $$b_\min = \max\left\{ρ_⟂, λ_{dB} \right\}.$$ The [impact_parameter()] function will calculate both values of $b_\min$ and choose for you. Let's estimate $b_\min$ and $b_\max$ for electron-proton collisions in the solar corona.

In [67]:
#Create a list of input values for the impact_parameter() function
solar_corona = {
    "T": 1e6 * u.K,
    "n_e": 1e9 * u.cm**-3,
    "species": ["e-", "p+"],
}

bmin, bmax = impact_parameter(**solar_corona)

print(f"{bmin = :.2e}")
print(f"{bmax = :.2e}")

#use the same values for the coulomb logarithm calculation
Coulomb_logarithm(**solar_corona)

bmin = 1.05e-11 m
bmax = 2.18e-03 m


19.150697742645406

So $\ln\Lambda$ is $\sim 20$. Now we are ready to calculate the collision frequency.

In [68]:
n = 1e9 * u.cm**-3
Coulomb_log = Coulomb_logarithm(**solar_corona) * u.dimensionless_unscaled

#Inputs: the two particle types, the drift velocity (can be 0 since we use the Lorentz_collision_frequency attribute), 
#the temperature and densities of the two particles, which we assume are the same in this case, and the Coulomb logarithm we calculated
frequencies = coll.frequencies.MaxwellianCollisionFrequencies(
    "e-", "p+", v_drift=0 * u.m / u.s, T_a=T_corona,n_a=n,n_b=n, T_b=T_corona, Coulomb_log=Coulomb_log
)
frequencies.Lorentz_collision_frequency

<Quantity 92.41654907 Hz>

Whether or not this collision rate contributes to heating depends on how this rate compares to the other motions in the plasma. For example, the plasma frequency (REF from Ch 3) and the gyrofrequency (REF from Ch 3). Let's calculate them and compare:

In [69]:
from plasmapy.formulary import plasma_frequency
plasma_frequency(n, ['e-','p+']).to("Hz", equivalencies=u.dimensionless_angles())

<Quantity [1.78398637e+09, 4.16329453e+07] Hz>

In [70]:
from plasmapy.formulary import gyrofrequency
B = 50 * u.G #magnetic field strength in corona, for gyrofrequency calc
gyrofrequency(B, ['e-','p+']).to("Hz", equivalencies=u.dimensionless_angles())

<Quantity [8.79410005e+08, 4.78941658e+05] Hz>

So the $\omega_p$ and $\omega_c$ and both much larger than the collision frequency, particularly for electrons. Effects from the collisions are then negligible, and the corona is considered a collisionless plasma.

The solar wind is even more collisionless.

In [71]:
solar_wind = {
    "T": 6e5 * u.K,
    "n_e": 25 * u.cm**-3,
    "species": ["e-", "p+"],
}

bmin, bmax = impact_parameter(**solar_wind)

print(f"{bmin = :.2e}")
print(f"{bmax = :.2e}")

Coulomb_log = Coulomb_logarithm(**solar_wind)* u.dimensionless_unscaled
print(f"{Coulomb_log = :.2e}")

T_wind = 6*10**5*u.K
n_wind = 25 * u.cm**-3

frequencies = coll.frequencies.MaxwellianCollisionFrequencies(
    "e-", "p+", v_drift=0 * u.m / u.s, T_a=T_wind,n_a=n_wind,n_b=n_wind, 
    T_b=T_wind, Coulomb_log=Coulomb_log
)
frequencies.Lorentz_collision_frequency

bmin = 1.39e-11 m
bmax = 1.07e+01 m
Coulomb_log = 2.74e+01


<Quantity 7.10396736e-06 Hz>

In other words, collisions happen very, very rarely in the solar wind.